## Некоторые практические рекомендации

### Какой ML алгоритм в общем случае выбрать: RF или GBDT?

Из [практики соревнований по машинному обучению ✏️[blog]](https://www.kaggle.com/code/kashnitsky/topic-10-gradient-boosting/notebook) известно, что использование градиентного бустинга для решения задач с табличными данными даёт возможность получить максимально качественное решение. Тем не менее, следует понимать, что это гипотетическое лучшее качество решения достигается путём сложного подбора параметров и финальной настройки модели.

В качестве короткой практической рекомендации можно сформулировать следующий пайплайн для решения задачи на реальных табличных данных:

1. Обучить на имеющихся данных "простую" модель случайного леса, оценку её качества взять за отправную точку при дальнейшей работе.
1. Провести эксперименты по отбору и конструированию признаков (feature engineering) (благодаря гипотетической большей скорости обучения случайного леса, разумно проводить это исследование именно с этой моделью).
1. Повторять эксперименты с feature engineering, подбирая оптимальные основные гиперпараметры случайного леса (об этом ниже).
1. Отобрав лучшие варианты наборов признаков из экспериментов с решением на случайных лесах, использовать их при обучении модели одной из реализаций градиентного бустинга (если в данных доминируют категориальные признаки, то в первую очередь следует воспользоваться реализацией бустинга от CatBoost, в противном случае можно воспользоваться любой популярной реализацией — CatBoost, XGBoost, LightGBM).
1. Произвести настройку гиперпараметров модели (об этом ниже).
1. Если не получилось достичь желаемого качества решения, следует вернуться к экспериментам с feature engineering, но уже сразу проводить их для модели бустинга. Вернуться к этапу подбора гиперпараметров.

### Какие деревья решений использовать в качестве элементов случайного леса, а на каких строить градиентный бустинг?

Как мы уже успели установить выше, справедливо разложение:

```ошибка предсказания модели = bias + variance```

и имеет место явление bias-variance tradeoff, принципиально не позволяющее одинаково сильно минимизировать оба вклада в ошибку предсказания модели.

* Ансамблирование в виде бэггинга, использующееся при построении RF, заточено на уменьшение `variance` путём "усреднения" ответов элементарных предсказателей. Использование бэггинга существенно не позволяет улучшить `bias` — этот показатель наследуется от базового эстиматора. Таким образом, **при построении случайного леса следует взять относительно глубокие деревья решений** (можно говорить о глубине в 6–10 ветвлений), обладающие собственным низким показателем `bias`.

* Ансамблирование в виде бустинга, наоборот, в первую очередь заточено на улучшение показателя `bias` итоговой модели путём взаимного улучшения большого числа *стабильных, но слабых* эстиматоров (то есть обладающих высоким показателем `bias` и низким `variance`). Таким образом, **при построении градиентного бустинга над деревьями решений следует взять в качестве базовой модели неглубокие деревья** (вплоть до решающих пней — деревьев решений с единственным ветвлением).

### У RF значительно меньше гиперпараметров, чем у GBDT. Но как их выбрать?

Фактически, все реализации алгоритма случайного леса будут содержать три важных гиперпараметра:

* **Глубина деревьев**.

Мы ожидаем, что каждое из деревьев в составе леса обладает достаточной обобщающей способностью, чтобы быть принципиально способным самостоятельно предсказывать данные. То есть во время подбора гиперпараметров случайного леса данный параметр стоит постепенно увеличивать, контролируя качество на тренировочной и валидационной выборках — слишком глубокие деревья могут привести к переобучению модели.

* **Количество деревьев** в лесу.

Бэггинг деревьев решений позволяет нам минимизировать `variance` итоговой модели, но понятно, что ресурс для такого улучшения не безграничен. Количество деревьев в лесу можно постепенно увеличивать до тех пор, пока это не перестанет приводить к существенному улучшению качества предсказания модели.


* Какая **доля признаков** подаётся отдельным деревьям решений в составе случайного леса.

Подавая на вход одиночным деревьям решений б**о**льшее число признаков, мы делаем их более "сильными предсказателями", но взамен увеличиваем их скоррелированность (тем самым нарушаем процедуру бэггинга). Для конкретных данных положение оптимума можно наверняка установить только полным перебором параметра, но общей практикой является использование трети от общего числа свойств для задачи регрессии и округленного вниз квадратного корня из числа свойств для задач классификации.


### Какие гиперпараметры GBDT реализаций следует подбирать в первую очередь?

Современные реализации GBPD содержат крайне широкий список доступных для настройки параметров. Первичное знакомство с длинными списками параметров может оказаться дезориентирующим, поэтому мы приведём короткий список наиболее существенных гиперпараметров, которые повлияют на успех обучения модели.

Среди таких наиболее важных параметров выделим два класса:
* параметры, влияющие на контроль переобучения модели;
* параметры, определяющие скорость процесса обучения модели.

Названия параметров в приведённой ниже таблице являются ссылками на актуальную версию документации:

| param type            | CatBoost                         | XGBoost                                        | LightGBM                                                      |
|-----------------------|----------------------------------|------------------------------------------------|---------------------------------------------------------------|
| overfitting control   | [`learning_rate`](https://catboost.ai/en/docs/concepts/parameter-tuning#learning-rate)<br>[`depth`](https://catboost.ai/en/docs/concepts/parameter-tuning#tree-depth)<br>[`l2_leaf_reg`](https://catboost.ai/en/docs/concepts/parameter-tuning#l2-reg) | [`eta`](https://xgboost.readthedocs.io/en/stable/parameter.html#:~:text=%EF%83%81-,eta,-%5Bdefault%3D0.3%2C%20alias)<br>[`max_depth`](https://xgboost.readthedocs.io/en/stable/parameter.html#:~:text=range%3A%20%5B0%2C%E2%88%9E%5D-,max_depth,-%5Bdefault%3D6%5D)<br>[`min_child_weight`](https://xgboost.readthedocs.io/en/stable/parameter.html#:~:text=range%3A%20%5B0%2C%E2%88%9E%5D-,min_child_weight,-%5Bdefault%3D1%5D) | [`learning_rate`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#learning_rate)<br>[`max_depth`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#max_depth)<br>[`num_leaves`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#num_leaves)<br>[`min_data_in_leaf`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#min_data_in_leaf) |
| speed of the training | [`rsm`](https://catboost.ai/en/docs/references/training-parameters/common#rsm)<br>[`iterations`](https://catboost.ai/en/docs/references/training-parameters/common#iterations)                | [`colsample_bytree`](https://xgboost.readthedocs.io/en/stable/parameter.html#:~:text=support%20uniform%20sampling.-,colsample_bytree,-%2C%20colsample_bylevel%2C)<br>[`subsample`](https://xgboost.readthedocs.io/en/stable/parameter.html#:~:text=range%3A%20%5B0%2C%E2%88%9E%5D-,subsample,-%5Bdefault%3D1%5D)<br>[`n_estimators`](https://xgboost.readthedocs.io/en/stable/python/python_api.html#:~:text=for%20XGBoost%20regression.-,Parameters,n_estimators%20(int),-%E2%80%93%20Number%20of%20gradient)  | [`feature_fraction`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#feature_fraction)<br>[`bagging_fraction`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#bagging_fraction)                          |


